In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dense

In [5]:
def create_subway_network_data():
    """
    随机创建的交通数据
    这个函数里创造的数据请根据实际GIS数据或者图像进行替换！！！

    现在我们创造这个数据集用来表示:
    - 地铁站的位置（x,y坐标）
    - 地铁线路的连接关系
    - 地铁站的属性（换乘站、终点站等）
    """

    # 假设全市所有地铁线路经过的全部站点一共是20个地铁站
    n_stations = 20

    # 创建站点数据
    stations = pd.DataFrame({
        'station_id': range(1, n_stations + 1),
        'x_coord': np.random.uniform(0, 100, n_stations),
        'y_coord': np.random.uniform(0, 100, n_stations),
        'is_transfer': np.random.choice([0, 1], n_stations, p=[0.8, 0.2]), # 这一站能不能换乘其他线路或者公交
        'is_terminal': np.random.choice([0, 1], n_stations, p=[0.9, 0.1]), # 是不是起始终点站
        'n_exits': np.random.randint(1, 6, n_stations), # 这个站点几个出口
        'has_mall': np.random.choice([0, 1], n_stations, p=[0.7, 0.3]), # 有无购物中心
        'district_population': np.random.uniform(50000, 500000, n_stations), # 地铁站所在片区的人口数量，这里假设片区的人口数量是5w-50w
        'business_area': np.random.uniform(0, 1, n_stations), # 地铁站周边商业区面积
        'residential_area': np.random.uniform(0, 1, n_stations) # 地铁站周边住宅区占比
    })

    # 假设有4条地铁线，每条线经过的站点如下
    lines = [
        [1, 2, 3, 4, 5],  # 线路1
        [6, 7, 8, 9, 10],  # 线路2
        [11, 12, 13, 14, 15],  # 线路3
        [16, 17, 18, 19, 20]  # 线路4
    ]

    # 添加部分交叉线路（这里简单演示多条线路的交叉站点）
    lines[1].append(5)  # 线路2也包含站点5
    lines[2].append(8)  # 线路3也包含站点8
    lines[0].append(15)  # 线路1也包含站点15

    # 用位掩码存储线路信息
    # 线路1对应位掩码0001（比如站点1就是0001）
    # 线路2对应位掩码0010
    # 线路3对应位掩码0100
    # 线路4对应位掩码1000
    # 如果站点属于线路1和线路3，则其线路掩码为0101（站点5）

    # 初始化line_mask列
    stations['line_mask'] = 0

    # 对线路和站点进行标号
    for line_id, line in enumerate(lines, 1): # 因为地铁一般从1号线开始
        line_bit = 1 << (line_id - 1)  # 从0001开始，线路2就让它的掩码中的1向左位移2-1=1个位置，变成0010
        for station_id in line:
            # 先从stations表中筛选出station_id等于指定值的行，然后获取这些行的'line_mask'列，最后取出结果的第一个值
            # 由于在一个表中，站点ID应该是唯一的，所以筛选结果应该只有一行，也就是values[0]
            current_mask = stations.loc[stations['station_id'] == station_id, 'line_mask'].values[0]
            stations.loc[stations['station_id'] == station_id, 'line_mask'] = current_mask | line_bit # 使用位运算OR来添加线路到掩码中

        # 添加线路列表，把line_mask列中的掩码转换成列表存储线路编号
        stations['line_list'] = stations['line_mask'].apply(
            lambda mask: [i + 1 for i in range(4) if (mask & (1 << i)) > 0] # 把第i位(0-3)转换为1-4的编号
        )

        # 将站点信息转换为224x224的图像格式
        image_size = 224

        # 使用7个通道的图像表示:
        # 通道0-3: 4条线路的站点位置
        # 通道4: 站点属性（换乘站、终点站）
        # 通道5: 周边环境信息
        # 通道6: 线路连接
        subway_image = np.zeros((image_size, image_size, 7))

        # 归一化坐标到图像大小
        stations['img_x'] = (stations['x_coord'] / 100 * image_size).astype(int)
        stations['img_y'] = (stations['y_coord'] / 100 * image_size).astype(int)

        # 在图像上标记站点
        for _, station in stations.iterrows():
            x, y = station['img_x'], station['img_y']
            line_mask = station['line_mask']

            # 遍历所有可能的线路
            for line_id in range(4):  # 4条线路，索引0-3
                # 检查站点是否属于当前线路
                if (line_mask & (1 << line_id)) > 0:
                    # 在对应线路的通道上标记站点
                    subway_image[y, x, line_id] = 1.0

            # 站点属性通道（这里简单粗暴地让转运点和起终点各占一半影响，请参考文献和不同城市情况修改权重）
            subway_image[y, x, 4] = station['is_transfer'] * 0.5 + station['is_terminal'] * 0.5

            # 周边信息通道（同上注意修改权重）
            subway_image[y, x, 5] = (
                    station['district_population'] / 500000 * 0.4 +
                    station['business_area'] * 0.3 +
                    station['residential_area'] * 0.3
            )

            # 我们考虑每一个站点产生一个有影响力的片区
            # 分别遍历地铁站点四周和地铁站点本身一共9个坐标，生成3x3的区域
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    nx, ny = x + dx, y + dy
                    # 考虑到城市边缘的站点影响力一般没有城市内部的站点影响力大，并且很少有这样的站点
                    # 所以这里不使用padding了，超过地图边缘的一律不再扩展，仅使用地铁站本身的1x1坐标
                    if 0 <= nx < image_size and 0 <= ny < image_size:
                        # 对每个线路通道扩展站点
                        for line_id in range(4):
                            if (line_mask & (1 << line_id)) > 0:
                                # 站点坐标影响力是其本身，站点周身8个坐标影响力取0.7，代表站点影响力呈辐射状
                                subway_image[ny, nx, line_id] = max(subway_image[ny, nx, line_id], 0.7)

                        # 站点属性
                        subway_image[ny, nx, 4] = max(
                            subway_image[ny, nx, 4],
                            (station['is_transfer'] * 0.5 + station['is_terminal'] * 0.5) * 0.7
                        )

                        # 周边信息
                        subway_image[ny, nx, 5] = max(
                            subway_image[ny, nx, 5],
                            (station['district_population'] / 500000 * 0.4 +
                             station['business_area'] * 0.3 +
                             station['residential_area'] * 0.3) * 0.7
                        )

        # 创建连接线
        for line_id, line in enumerate(lines):
            for i in range(len(line) - 1):
                # 跳过连接到已处理站点的线段，避免重复绘制
                if i > 0 and line[i] in line[:i]:
                    continue

                # 把线路上的地铁站两两连接
                station1 = stations[stations['station_id'] == line[i]].iloc[0]
                station2 = stations[stations['station_id'] == line[i + 1]].iloc[0]

                x1, y1 = station1['img_x'], station1['img_y']
                x2, y2 = station2['img_x'], station2['img_y']

                # 使用Bresenham算法绘制线条
                steps = max(abs(x2 - x1), abs(y2 - y1))
                if steps > 0:
                    x_step = (x2 - x1) / steps
                    y_step = (y2 - y1) / steps

                    for step in range(steps + 1):
                        x = int(x1 + step * x_step)
                        y = int(y1 + step * y_step)

                        if 0 <= x < image_size and 0 <= y < image_size:
                            subway_image[y, x, 6] = 1.0  # 在线路连接通道上绘制线路

        return subway_image, stations, lines

In [6]:
def darknet_block(x, filters, kernel_size=3, strides=1):
    """
    YOLO的基本构建块，包含卷积、批归一化和LeakyReLU激活

    参数:
    - x: 输入张量
    - filters: 卷积层过滤器数量
    - kernel_size: 卷积核大小
    - strides: 卷积步长

    返回:
    - 处理后的张量
    """
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    # 防止负值特征丢失
    x = LeakyReLU(alpha=0.1)(x)
    return x

In [7]:
def build_yolo_feature_extractor(input_shape=(224, 224, 7)):
    """
    构建基于YOLO架构的特征提取器，用于提取地铁网络的空间特征
    训练参数请自行修改喔~

    参数:
    - input_shape: 输入数据的形状，默认为(224, 224, 7)

    返回:
    - 地铁网络特征提取模型
    """
    inputs = Input(shape=input_shape)

    # 第一个块
    x = darknet_block(inputs, 16)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 第二个块
    x = darknet_block(x, 32)
    x = darknet_block(x, 16, kernel_size=1)  # 1x1 卷积减少通道数
    x = darknet_block(x, 32)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 第三个块
    x = darknet_block(x, 64)
    x = darknet_block(x, 32, kernel_size=1)
    x = darknet_block(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 第四个块
    x = darknet_block(x, 128)
    x = darknet_block(x, 64, kernel_size=1)
    x = darknet_block(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 第五个块
    x = darknet_block(x, 256)
    x = darknet_block(x, 128, kernel_size=1)
    x = darknet_block(x, 256)

    # 全局平均池化替代全连接层，减少参数
    x = GlobalAveragePooling2D()(x)

    # 最终特征向量
    outputs = Dense(128, activation='relu', name='subway_network_features')(x)

    # 创建模型
    model = Model(inputs=inputs, outputs=outputs, name='yolo_subway_feature_extractor')

    return model

In [8]:
def extract_subway_features(subway_image):
    """
    使用YOLO特征提取器提取地铁网络特征
    PS：在实际应用中，请使用预训练的YOLO权重或在相关数据集上训练模型，这里没有真实数据所以我只用了随机初始化的权重来做

    参数:
    - subway_image: 地铁网络图像

    返回:
    - 提取的特征向量
    """
    # 构建特征提取器
    feature_extractor = build_yolo_feature_extractor(input_shape=subway_image.shape)

    # 扩展维度以匹配批处理维度
    subway_image_batch = np.expand_dims(subway_image, axis=0)

    # 提取特征
    features = feature_extractor.predict(subway_image_batch)

    return features[0]  # 返回第一个样本的特征（去掉批处理维度）

In [9]:
def main():
    # 创建示例地铁网络数据
    subway_image, stations, lines = create_subway_network_data()

    print("地铁网络图像形状:", subway_image.shape)
    print(f"共有 {len(stations)} 个站点和 {len(lines)} 条线路")

    # 显示线路信息
    for i, line in enumerate(lines, 1):
        print(f"线路 {i}: {line}")

    # 显示部分站点的线路信息
    print("\n站点所属线路信息:")
    for _, station in stations.iterrows():
        if len(station['line_list']) > 1:  # 只显示属于多条线路的站点
            print(f"站点 {station['station_id']}: 线路 {station['line_list']} (掩码: {station['line_mask']})")

    # 提取地铁网络特征
    subway_features = extract_subway_features(subway_image)

    print("\n提取的地铁网络特征形状:", subway_features.shape)
    print("地铁网络特征（前10个）:", subway_features[:10])

    # 显示特征提取器模型摘要
    model = build_yolo_feature_extractor()
    model.summary()

    # print("\n改进的编码方案优势:")
    # print("1. 支持站点属于多条线路，更符合实际地铁网络的复杂性")
    # print("2. 使用位掩码高效存储线路信息，方便位运算和查询")
    # print("3. 每条线路仍然在单独的通道上编码，保持了线路特征的区分性")
    # print("4. 特征提取效率高，适合大型地铁网络")

In [10]:
if __name__ == "__main__":
    main()

地铁网络图像形状: (224, 224, 7)
共有 20 个站点和 4 条线路
线路 1: [1, 2, 3, 4, 5, 15]
线路 2: [6, 7, 8, 9, 10, 5]
线路 3: [11, 12, 13, 14, 15, 8]
线路 4: [16, 17, 18, 19, 20]

站点所属线路信息:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 583ms/step

提取的地铁网络特征形状: (128,)
地铁网络特征（前10个）: [0.         0.         0.00030624 0.         0.         0.
 0.0010142  0.00086003 0.00073436 0.        ]


Model: "yolo_subway_feature_extractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 7)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 224, 224, 16)        │           1,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 224, 224, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 224, 224, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 112, 112, 32)        │           4,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 112, 112, 16)        │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 112, 112, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_15 (LeakyReLU)           │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 112, 112, 32)        │           4,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_16 (LeakyReLU)           │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 56, 56, 64)          │          18,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 56, 56, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_17 (LeakyReLU)           │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 56, 56, 32)          │           2,0

 Total params: 865,648 (3.30 MB)

 Trainable params: 863,216 (3.29 MB)

 Non-trainable params: 2,432 (9.50 KB)